In [1]:
import pandas as pd
import numpy as np
import os
import operator
import requests
from datetime import date

In [2]:
df = pd.read_csv("Data/Yellowpages_27-04-2020.csv")
#df2 = pd.read_csv("Data/Googlemaps_20-04-2020.csv")

In [10]:
df = pd.concat([df, df2], sort=False)

NameError: name 'df2' is not defined

In [3]:
def merge_sheets(path = "Data/"):
    files = os.listdir(path)
    
    for file in files:
        initial = True
        if file[-3:] == "csv":
            if initial == True:
                df = pd.read_csv(path + file)
                initial = False
            else:
                this_df = pd.read_csv(path + file)
                df = pd.concat([df, this_df], sort=False)
    return df

def cleanup(df):
    df = df.drop(labels="Unnamed: 0", axis=1) 
            
    df = df.reset_index()
    df = df.drop(labels="index", axis=1)
    
    for i in range(0, len(df)):
        # only applies for google maps!
        #if df.iloc[i, 1][-11:] == ", Australia":
        #    df.iloc[i, 1] = df.iloc[i, 1][:-11]
            
        if df["Category"][i].find("-") != -1:
            df["Category"][i] = df["Category"][i][:df["Category"][i].find("-")]
            
        tel = str(df.Phone[i])
        if tel[0] == "1":
            tel = "+6" + tel
        else:
            bracket_o = tel.find("(")
            if bracket_o != -1:
                tel = tel[:bracket_o] + tel[bracket_o+2:]
                bracket_c = tel.find(")")
                if bracket_c != -1:
                    tel = tel[:bracket_c] + tel[bracket_c+1:]
                if str(tel[:3]) != "+61":
                    tel = "+61" + tel
            elif tel[0] == "0":
                tel = "+61" + tel[1:]
        df.Phone[i] = tel
            
    return df

def get_zip(df):
    zip_codes = list()
    state = list()
    for i in range(0, len(df.Name)):
        try:
            zip_code = df.Adress[i][-4:]
            zip_codes.append(zip_code)
        except:
            zip_codes.append("N/V")
            
        try:
            intzip_code = int(zip_code)
            if intzip_code >= 1000 and intzip_code < 3000:
                state.append("New South Wales")
            elif (intzip_code >= 3000 and intzip_code < 4000) or (intzip_code >= 8000 and intzip_code < 9000):
                state.append("Victoria")
            elif (intzip_code >= 4000 and intzip_code < 5000) or (intzip_code >= 9000 and intzip_code < 10000):
                state.append("Queensland")
            elif intzip_code >= 5000 and intzip_code < 6000:
                state.append("South Australia")
            elif intzip_code >= 6000 and intzip_code < 7000:
                state.append("Western Australia")
            elif intzip_code >= 7000 and intzip_code < 8000:
                state.append("Tasmania")
            elif intzip_code >= 800 and intzip_code < 1000:
                state.append("Nothern Australia")
            else:
                state.append("N/V")
        except:
            state.append("N/V")
            
            
    df["zip"] = zip_codes
    df["state"] = state
    return df

"""    for name in duplicates["names"]:
        part_df = df[df.Name == name]
        for i in range(0, len(part_df)):
            try:
                if len(part_df[part_df.zip == part_df.zip[i]]) > 1:
                    print("zip here.")
            except:
                pass"""

def find_duplicates(df):
    cutOff = int((len(df)/2) + 1)
    duplicates = list()
    for i in range(0, cutOff):
        if len(df[df.Name == df.Name[i]]) > 1:
            duplicates.append(df.Name[i])
    
    return duplicates

def handle_duplicates(df, duplicates):
    columns = ["Adress", "GeoLocation", "Phone", "Website", "Email"]
    for i in range(0, len(duplicates)):
        part_df = df[(df.Name == df.Name[i]) & (df.zip == df.zip[i])]

        best_entry = dict()
        for column in columns:
            for i, entry in enumerate(part_df[column]):
                   if entry != "N/V" and entry != "":
                        if i not in best_entry.keys():
                            best_entry[i] = 1
                        else:
                            best_entry[i] = best_entry[i] + 1

        best_entry = max(best_entry.items(), key=operator.itemgetter(1))[0]
        empty_columns = list()


        for column in columns:
            try:
                if part_df[column][best_entry] == "N/V":
                    empty_columns.append(column)
            except:
                pass

        if len(empty_columns) != 0:
            for i in range(0, len(part_df)):
                if i != best_entry:
                    for column in empty_columns:
                        if part_df[column][i] != "N/V" and part_df[column][i] != "":
                              part_df[column][best_entry] =  part_df[column][i]

        df = df.drop(index=part_df.index)
        df = df.append(part_df.iloc[best_entry])
    return df

In [133]:
#df = merge_sheets()

In [6]:
df = cleanup(df)

/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
df = get_zip(df)

In [18]:
duplicates = find_duplicates(df)

In [8]:
df.tail(20)

,Name,Adress,GeoLocation,Phone,Website,Category,Email,Crops,Source,DateUpdated,DateChecked,zip,state
79434,Australian Organic Network,"1030 Osmaston Rd, Deloraine TAS 7304",NaN,+61409 139 027,http://www.australianorganicnetwork.com.au/,Organic Products,australianorganicnetwork@gmail.com,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7304,Tasmania
79435,Bellamy's Organic,Launceston TAS 7250,NaN,+61800 010 460,http://www.bellamysorganic.com.au/,Organic Products,marketing@bellamysorganic.com.au,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7250,Tasmania
79436,ecoHaven,"27 Murray St, Hobart TAS 7000",NaN,+613 6223 7620,http://ecohaven.com.au/,Organic Products,shop@ecohaven.com.au,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7000,Tasmania
79437,The Wholesome Health Bug,"6/79 Main St, Huonville TAS 7109",NaN,+613 6264 2428,https://www.facebook.com/wholesomehealthbug,Organic Products,N/V,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7109,Tasmania
79438,New Town Greenstore,"134 New Town Rd, New Town TAS 7008",NaN,+613 6278 1306,N/V,Organic Products,N/V,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7008,Tasmania
79439,Beauty & The Bees,"Shop 209C Centrepoint Shopping Ctr Murray St, ...",NaN,+613 6236 9977,N/V,Organic Products,N/V,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7000,Tasmania
79440,Source Community Wholefoods,"12 French St, Sandy Bay TAS 7005",NaN,+613 6224 0055,N/V,Organic Products,N/V,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7005,Tasmania
79441,Just Tasmanian,"1567 Channel Hwy, Margate TAS 7054",NaN,+61419 874 461,N/V,Organic Products,N/V,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7054,Tasmania
79442,Eatem Organic Foods,"1333 Lonnavale Rd, Lonnavale TAS 7109",NaN,+613 6266 0330,N/V,Organic Products,N/V,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7109,Tasmania
79443,East West Organics,"127 Poplar Parade, Youngtown TAS 7249",NaN,+613 6343 6459,N/V,Organic Products,N/V,NaN,https://www.yellowpages.com.au/search/listings...,27/04/2020,-,7249,Tasmania


In [5]:
len(df)

79454

In [21]:
df_noDuplicates = handle_duplicates(df, duplicates)

KeyError: 1

In [12]:
df_noDuplicates.head()

,Name,Adress,GeoLocation,Phone,Website,Email,Source,DateUpdated,DateChecked,zip,state
43,Babylon Farm,Murrumbateman ACT 2582,NaN,0477 330 598,http://www.babylonfarm.com.au/,visit@babylonfarm.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,2582,NSW
44,Care Outreach,"35 Dacmar Rd, Coolum Beach QLD 4573",NaN,(07) 5471 7636,https://careoutreach.com.au/,info@careoutreach.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,4573,Queensland
45,Biomix,"980 Two Tree Rd, Carag Carag VIC 3623",NaN,0488 993 066,https://www.biomix.com.au/,info@biomix.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,3623,Victoria
55,K S Benning & Sons,"24B Bosworth Rd, Woolgoolga NSW 2456",NaN,(02) 6654 8775,http://www.ksbenning.com.au/,admin@ksbenning.com.au,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,2456,NSW
56,CN & BL McGillivray,N/V,NaN,(03) 5593 2967,http://www.cnandblmcgillivray.com.au/,col.bev3260@gmail.com,https://www.yellowpages.com.au/search/listings...,20/04/2020,-,N/V,N/V


In [13]:
def try_websites(df):
    for i in range(0, len(df.Name)):
        if i % 100 == 0:
            print(i)
        try:
            request = requests.get(str(df.Website[i]))
        except:
            df.Website[i] = "N/V"
    return df

In [9]:
df.to_excel(f"Data/Data_{date.today().strftime('%d-%m-%Y')}.xlsx")

/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:912: UserWarning: Ignoring URL 'https://www.google.com/maps/uv?hl=en&pb=!1s0x6b131352bb4183d7%3A0x6b83de7f4cf80813!2m22!2m2!1i80!2i80!3m1!2i20!16m16!1b1!2m2!1m1!1e1!2m2!1m1!1e3!2m2!1m1!1e5!2m2!1m1!1e4!2m2!1m1!1e6!3m1!7e115!4shttps%3A%2F%2Flh5.googleusercontent.com%2Fp%2FAF1QipM3VsHBMFddRRJaOXoekiVXOTG5vxr3gnpn7fUV%3Dw354-h200-k-no!5sjason%20kelly%20barrack%20heights%20-%20Google%20Search!15sCAQ&imagekey=!1e10!2sAF1QipM3VsHBMFddRRJaOXoekiVXOTG5vxr3gnpn7fUV&sa=X&ved=2ahUKEwi6vrixl__jAhXX6nMBHd6NC-gQoiowE3oECA8QBg' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:912: UserWarning: Ignoring URL 'http://www.sahealth.sa.gov.au/wps/wcm/connect/public+content/sa+health+internet/health+services/hospitals+and+health+services+-+country+south+australia/fleurieu+peninsula+k

/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=chicken&eventType=pagination&locationClue=VIC&pageNumber=18&referredBy=www.yellowpages.com.au' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=chicken&eventType=pagination&locationClue=VIC&pageNumber=19&referredBy=www.yellowpages.com.au' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=chicken&eventType=pagination&locationClue=VIC&pageNumber=20&referredBy=www.

/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=horse&eventType=pagination&locationClue=VIC&pageNumber=5&referredBy=www.yellowpages.com.au' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=horse&eventType=pagination&locationClue=VIC&pageNumber=6&referredBy=www.yellowpages.com.au' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=horse&eventType=pagination&locationClue=VIC&pageNumber=7&referredBy=www.yellowpag

/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=wool&eventType=pagination&locationClue=NSW&pageNumber=5&referredBy=www.yellowpages.com.au' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=wool&locationClue=QLD&pageNumber=1&referredBy=www.yellowpages.com.au&&eventType=pagination' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=wool&locationClue=VIC&pageNumber=1&referredBy=www.yellowpages.com.au&&eventType=pa

/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=asian+vegetable&eventType=pagination&locationClue=VIC&pageNumber=19&referredBy=www.yellowpages.com.au' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=asian+vegetable&eventType=pagination&locationClue=VIC&pageNumber=20&referredBy=www.yellowpages.com.au' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
/Users/maximilianwitte/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:920: UserWarning: Ignoring URL 'https://www.yellowpages.com.au/search/listings?clue=asian+vegetable&eventType=pagination&locationClue=VIC&pageN